In [1]:
!pip install rasterio
!pip install mercantile
import numpy as np
import pandas as pd
import imageio
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import mercantile

In [2]:
light_pollution = np.zeros((65536,65536), dtype=np.float16)

In [3]:
wa2015 = rio.open('World_Atlas_2015.tif')

In [4]:
(xTileMax,yTileMax,zTileMax) = mercantile.tile(*(wa2015.transform * (wa2015.width, wa2015.height)) + (16,))
print(xTileMax)
print(yTileMax)

65535
46502


In [5]:
wa_z16 = np.zeros((yTileMax+1, xTileMax+1), dtype=np.float32)

In [6]:
bottom_right_pixel = mercantile.bounds(xTileMax,yTileMax,zTileMax)
upper_left_pixel = mercantile.bounds(0,0,zTileMax)
bottom_right_pixel

LngLatBbox(west=179.9945068359375, south=-59.996239340364475, east=180.0, north=-59.99349233206086)

In [7]:
dst_transform, dst_width, dst_height = calculate_default_transform(
    wa2015.crs,
    {'init':'EPSG:3857'},
    wa2015.width,
    wa2015.height,
    left=upper_left_pixel.west,
    right=bottom_right_pixel.east,
    top=upper_left_pixel.north,
    bottom=bottom_right_pixel.south,
    dst_width=xTileMax+1,
    dst_height=yTileMax+1
)

In [8]:
reproject(
    wa2015.read(1),
    wa_z16,
    src_transform=wa2015.transform,
    dst_transform=dst_transform,
    src_crs=wa2015.crs,
    dst_crs={'init': 'EPSG:3857'},
    resampling=Resampling.lanczos
)


(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 Affine(611.483179193371, 0.0, -20037508.342789248,
        0.0, -611.5068443825922, 20037508.342789277))

In [9]:
light_pollution[0:yTileMax+1,0:xTileMax+1] = wa_z16
wa_z16 = None
wa2015 = None

In [10]:
light_pollution[25334, 10480]

4.938

In [11]:
#imageio.imwrite(
#    './light-pollution.png',
#    np.clip(
#        np.round(30 * np.log1p( np.clip(light_pollution,0,32) * 1024, dtype=np.float16)),
#        0,
#        255
#    ).astype(np.uint8))

In [12]:
25 * np.log1p(np.array([0, 0.00174, 0.014, 0.688, 3, 10], dtype=np.float16) * 1024)

array([  0.      ,  25.581287,  68.254974, 163.97266 , 200.76024 ,
       230.85387 ], dtype=float32)

In [13]:
water_blue = np.array([192.0, 224.0, 255.0, 255.0])
white = np.array([255.0, 255.0, 255.0, 255.0])
grey_haze = np.array([204.0, 192.0, 170.0, 255.0])

In [14]:
world_water = np.zeros((65536,65536), dtype=np.float16)

terrain_water_color = np.array([153.0, 179.0, 204.0, 255.0])

for lon in range(256):
    for lat in range(256):
        image = imageio.imread(f"./terrain-background/8/{lon}/{lat}.png")
        iswater = np.linalg.norm(image - terrain_water_color, axis=2, ord=2) < 8.57
        world_water[(lat+0)*256:(lat+1)*256, (lon+0)*256:(lon+1)*256] = iswater

#imageio.imwrite('./world-water.png', world_water.astype(np.uint8) * 255)

In [15]:
def zoomout2x2(aint):
    return aint.reshape((aint.shape[0]//2,2,aint.shape[1]//2,2)).sum(axis=3).sum(axis=1) / 4


In [16]:
#mipmaps
light_zoomlevels = [0, 1, 2, 3, 4, 5, 6, light_pollution]
water_zoomlevels = [0, 1, 2, 3, 4, 5, 6, world_water]

for i in range(len(light_zoomlevels)-2,-1,-1):
    light_zoomlevels[i] = zoomout2x2(light_zoomlevels[i+1])
    water_zoomlevels[i] = zoomout2x2(water_zoomlevels[i+1])


In [17]:
def mcdm2_to_greyscale(a):
    return np.clip(
        np.round(
            30 * np.log1p( 1024 * np.clip(a,0,50) )
        ),
        0,
        255)


tile_size = 512

linear_inverse_rgb_levels = np.array([[1 - i/255, 1 - i/255, 1 - i/255, 1] for i in range(256)])
linear_alpha_levels = np.array([[1, 1, 1, i/255] for i in range(256)])

grey_haze_palette = ((linear_alpha_levels ** 0.75) * grey_haze).round().astype(np.uint8)
white_palette = (linear_inverse_rgb_levels * white).round().astype(np.uint8)
blue_palette = (linear_inverse_rgb_levels * water_blue).round().astype(np.uint8)

for z in range(len(light_zoomlevels)):
    for lon in range(light_zoomlevels[z].shape[0]//tile_size):
        for lat in range(light_zoomlevels[z].shape[0]//tile_size):
            light_tile = light_zoomlevels[z][lat*tile_size:(lat+1)*tile_size, lon*tile_size:(lon+1)*tile_size]
            water_tile = water_zoomlevels[z][lat*tile_size:(lat+1)*tile_size, lon*tile_size:(lon+1)*tile_size]
            light_tile_normalized = mcdm2_to_greyscale(light_tile).astype(np.uint8)
            water_tile_binarized = water_tile.round().astype(np.bool)
            dark_blue_marble = np.where(np.expand_dims(water_tile_binarized,-1),
                                        blue_palette[light_tile_normalized],
                                        white_palette[light_tile_normalized])
            grey_fade = grey_haze_palette[light_tile_normalized]
            imageio.imwrite(f'./tiles/globe-{z}-{lon}-{lat}.png', dark_blue_marble)
            imageio.imwrite(f'./tiles/greys-{z}-{lon}-{lat}.png', grey_fade)


In [18]:
for z in [8, 9, 10]:
    mini_tile_size = tile_size // (2 ** (z-len(light_zoomlevels)+1))
    print(f'mini tile size {mini_tile_size}')
    for lon in range(light_zoomlevels[-1].shape[0]//mini_tile_size):
        for lat in range(light_zoomlevels[-1].shape[0]//mini_tile_size):
            light_tile = light_zoomlevels[-1][lat*mini_tile_size:(lat+1)*mini_tile_size,
                                              lon*mini_tile_size:(lon+1)*mini_tile_size]
            water_tile = water_zoomlevels[-1][lat*mini_tile_size:(lat+1)*mini_tile_size,
                                              lon*mini_tile_size:(lon+1)*mini_tile_size]
            light_tile_normalized = mcdm2_to_greyscale(light_tile).astype(np.uint8)
            water_tile_binarized = water_tile.astype(np.bool)
            dark_blue_marble = np.where(np.expand_dims(water_tile_binarized,-1),
                                        blue_palette[light_tile_normalized],
                                        white_palette[light_tile_normalized])
            imageio.imwrite(f'./tiles/globe-{z}-{lon}-{lat}.png', dark_blue_marble)


mini tile size 256
mini tile size 128
mini tile size 64
